<a href="https://colab.research.google.com/github/erickrribeiro/ufam_nlp_01_2025/blob/main/custom_metrics/fase-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Obrigatório para testes no Google Colab
from google.colab import userdata

github_token = userdata.get('GITHUB_TOKEN')
!git clone https://{github_token}@github.com/erickrribeiro/ufam_nlp_01_2025.git
!mv ./ufam_nlp_01_2025/scripts/** ./

Cloning into 'ufam_nlp_01_2025'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 35.34 KiB | 786.00 KiB/s, done.


In [10]:
# !rm -rf ./ufam_nlp_01_2025

In [17]:
!unzip ./spider_data.zip -d spider

Archive:  ./spider_data.zip
   creating: spider/spider_data/
  inflating: spider/spider_data/dev_gold.sql  
  inflating: spider/__MACOSX/spider_data/._dev_gold.sql  
   creating: spider/spider_data/database/
  inflating: spider/__MACOSX/spider_data/._database  
  inflating: spider/spider_data/.DS_Store  
  inflating: spider/__MACOSX/spider_data/._.DS_Store  
  inflating: spider/spider_data/test_tables.json  
  inflating: spider/__MACOSX/spider_data/._test_tables.json  
  inflating: spider/spider_data/train_others.json  
  inflating: spider/__MACOSX/spider_data/._train_others.json  
  inflating: spider/spider_data/train_spider.json  
  inflating: spider/__MACOSX/spider_data/._train_spider.json  
  inflating: spider/spider_data/test.json  
  inflating: spider/__MACOSX/spider_data/._test.json  
  inflating: spider/spider_data/tables.json  
  inflating: spider/__MACOSX/spider_data/._tables.json  
  inflating: spider/spider_data/dev.json  
  inflating: spider/__MACOSX/spider_data/._dev.json

In [12]:
!pip install -U transformers==4.52.4 \
                trl==0.18.2 \
                peft==0.15.2 \
                datasets==3.6.0 \
                accelerate==1.8.0 \
                huggingface_hub==0.33.0 \
                fsspec==2025.3.0 \
                deepeval==3.1.6 --quiet

In [13]:
import datasets, transformers, trl, peft, accelerate, huggingface_hub, fsspec, deepeval

print("datasets:", datasets.__version__)
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("trl:", trl.__version__)
print("accelerate:", accelerate.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("fsspec:", fsspec.__version__)
print("deepeval:", deepeval.__version__)

datasets: 3.6.0
transformers: 4.52.4
peft: 0.15.2
trl: 0.18.2
accelerate: 1.8.0
huggingface_hub: 0.33.0
fsspec: 2025.3.0
deepeval: 3.1.6


- Funções úteis

In [18]:
from utils import (
  extract_sql,
  generate_schema_description,
  load_tables,
  load_few_shot,
  set_seed,
)

tables = load_tables("./spider/spider_data/tables.json")
schema_cache = {
  t["db_id"]: generate_schema_description(t) for t in tables
}
few_shot = load_few_shot()

# Fase 1: Estabelecimento do Baseline de Desempenho


## 1.1. Prompt Engineering:

A seguir, é definir um prompt com few-shot para a tarefa `Text-to-SQL`. O prompt deve contem 3 exemplos representativos extraídos do training split do Spider. Este template de prompt será utilizado em todas as avaliações de baseline.


In [19]:
import os
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

set_seed(42)
token = os.getenv('HF_TOKEN')

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=token,
                                          model_max_length=1024)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             token=token)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [20]:
from prompts import SYSTEM_PROMPT, USER_PROMPT_TEMPLATE

def build_prompt(
    schema: str,
    question: str,
    few_shot: list[dict] = [],
    system_prompt: str = SYSTEM_PROMPT,
) -> str:
    chat = [
      {
        "role": "system",
        "content": SYSTEM_PROMPT,
      },
    ]
    # Few-shot learning
    for example in few_shot:
      table = next(t for t in tables if t["db_id"] == example["db_id"])
      schema_i = generate_schema_description(table)
      chat.append({
        "role": "user",
        "content": USER_PROMPT_TEMPLATE.format(schema_i, example["question"]),
      })
      sql = example["query"]
      chat.append({
        "role": "assistant",
        "content": f"```sql {sql} ```",
      })

    chat.append({
      "role": "user",
      "content": USER_PROMPT_TEMPLATE.format(schema, question),
    })

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    return prompt

In [21]:
schema = """| department : Department_ID , Name , Creation , Ranking , Budget_in_Billions , Num_Employees |
| head : head_ID , name , born_state , age |
| management : department_ID , head_ID , temporary_acting |
Relationships:
management.head_ID = head.head_ID , management.department_ID = department.Department_ID
"""

question = "How many heads of the departments are older than 56 ?"
prompt = build_prompt(schema, question)
input = tokenizer(prompt, return_tensors="pt").to(device)

# Remove o prompt da saída
input_ids = input["input_ids"]
with torch.no_grad():
  output = model.generate(**input, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
  generated_tokens = output[0][input_ids.shape[1]:]

# output = model.generate(**input, max_new_tokens=256, do_sample=True, temperature=0.01)
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(response)

SELECT COUNT(*) FROM head
WHERE age > 56;


## 1.2 Execução da Avaliação

A seguir, o modelo base `mistralai/Mistral-7B-Instruct-v0.2` (não treinado) será submetido ao Spider dev split, utilizando o template de prompt definido na função `build_prompt`.

In [22]:
prompt = build_prompt(schema="fake_schema", question="fake_question", few_shot=few_shot)

print(prompt)

<s> [INST]  You are a helpful assistant who answers questions about database tables
by responding with SQL queries.

Schema:
| department : Department_ID , Name , Creation , Ranking , Budget_in_Billions , Num_Employees |
| head : head_ID , name , born_state , age |
| management : department_ID , head_ID , temporary_acting |
Relationships:
management.head_ID = head.head_ID , management.department_ID = department.Department_ID

Question: List the name, born state and age of the heads of departments ordered by age.
SQL:
 [/INST] ```sql SELECT name ,  born_state ,  age FROM head ORDER BY age ```</s> [INST] Schema:
| city : City_ID , Official_Name , Status , Area_km_2 , Population , Census_Ranking |
| farm : Farm_ID , Year , Total_Horses , Working_Horses , Total_Cattle , Oxen , Bulls , Cows , Pigs , Sheep_and_Goats |
| farm_competition : Competition_ID , Year , Theme , Host_city_ID , Hosts |
| competition_record : Competition_ID , Farm_ID , Rank |
Relationships:
farm_competition.Host_city_I

In [32]:
import os
import pandas as pd
from datasets import load_dataset
from tqdm.notebook import tqdm

ds = load_dataset("xlangai/spider")
tables = load_tables("./spider/spider_data/tables.json")

if not os.path.exists("./baseline_results.csv"):
  results = []
  i = 1
  for row in tqdm(ds["validation"]):
    db_id = row["db_id"]
    question = row["question"]
    gold_sql = row["query"]

    table = next(t for t in tables if t["db_id"] == db_id)
    schema_text = generate_schema_description(table)
    prompt = build_prompt(schema_text, question)

    input = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = input["input_ids"]
    with torch.no_grad():
      output = model.generate(
        **input,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id)
    generated_tokens = output[0][input_ids.shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    if i % 100 ==0:
      print("question:", question)
      print("gold_sql:", gold_sql)
      print("predicted_sql:", response)

    results.append({
      "db_id": db_id,
      "question": question,
      "gold_sql": gold_sql,
      "predicted_sql": extract_sql(response)
    })
    i+= 1
  df = pd.DataFrame(results)
else:
  df = pd.read_csv("baseline_results.csv")

df.head(n=5)

,db_id,question,gold_sql,predicted_sql
0,concert_singer,How many singers do we have?,SELECT count(*) FROM singer,SELECT COUNT(*) FROM singer;
1,concert_singer,What is the total number of singers?,SELECT count(*) FROM singer,SELECT COUNT(*) FROM singer;
2,concert_singer,"Show name, country, age for all singers ordere...","SELECT name , country , age FROM singer ORDE...","SELECT Name, Country, Age\nFROM singer\nORDER ..."
3,concert_singer,"What are the names, countries, and ages for ev...","SELECT name , country , age FROM singer ORDE...","SELECT singer.Name, singer.Country, singer.Age..."
4,concert_singer,"What is the average, minimum, and maximum age ...","SELECT avg(age) , min(age) , max(age) FROM s...","SELECT AVG(singer.Age) AS average_age, MIN(sin..."


## 1.3. Coleta de Dados

A seguir, será registrado o resultado da consulta SQL gerada para cada entrada utilizando o modelo `mistralai/Mistral-7B-Instruct-v0.2`. A métrica a ser reportada nesta fase é a contagem bruta de sucesso/falha.


In [40]:
import sqlite3

def execute_and_fetch(db_path, query):
  try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(query)
    results = cursor.fetchall()

    conn.close()
    return True, results, None
  except Exception as e:
      return False, None, str(e)


In [46]:
import os

execution_match = 0
total_evaluated = 0

results = df.to_dict(orient="records")
for row in tqdm(results):
    db_id = row["db_id"]
    db_path = os.path.join("spider/spider_data/database", db_id, f"{db_id}.sqlite")

    if not os.path.exists(db_path):
        row["execution_accuracy"] = "DB_NOT_FOUND"
        continue

    gold_query = row["gold_sql"]
    predicted_query = row["predicted_sql"]

    # Executa gold
    gold_success, gold_result, gold_error = execute_and_fetch(db_path, gold_query)

    # Executa predicted
    pred_success, pred_result, pred_error = execute_and_fetch(db_path, predicted_query)

    # Avaliação
    if gold_success and pred_success:
      total_evaluated += 1

      pred_set = set(pred_result)
      gold_set = set(gold_result)

      if pred_set == gold_set:
        execution_match += 1
        row["execution_accuracy"] = "MATCH"
      else:
        row["execution_accuracy"] = "MISMATCH"
    else:
      error_msg = ""
      if not gold_success:
        error_msg += f"GOLD_FAIL: {gold_error} | "
      if not pred_success:
        error_msg += f"PRED_FAIL: {pred_error}"
      row["execution_accuracy"] = error_msg.strip()

print(f"Total comparado (com sucesso em ambas): {total_evaluated}")
print(f"✔️ Matches: {execution_match}")
print(f"❌ Mismatches: {total_evaluated - execution_match}")

predictions = pd.DataFrame(results)

  0%|          | 0/1034 [00:00<?, ?it/s]

Total comparado (com sucesso em ambas): 664
✔️ Matches: 444
❌ Mismatches: 220


In [58]:
from pandas import option_context

with option_context('display.max_colwidth', 400):
  row = predictions.iloc[6]
  print(row)

db_id                                                                                                concert_singer
question                                     Show the name and the release year of the song by the youngest singer.
gold_sql                                     SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1
predicted_sql         SELECT s.Name AS singer_name, s.Song_release_year\nFROM singer s\nORDER BY s.Age ASC\nLIMIT 1
execution_accuracy                                                                                         MISMATCH
Name: 6, dtype: object


In [59]:
db_id = row.db_id
db_path = os.path.join("spider/spider_data/database", db_id, f"{db_id}.sqlite")

print("gold_sql:", execute_and_fetch(db_path, row.gold_sql))
print("predicted_sql: ", execute_and_fetch(db_path, row.predicted_sql))

gold_sql: (True, [('Love', '2016')], None)
predicted_sql:  (True, [('Tribal King', '2016')], None)
